In [1]:
import pywren, json, requests, time
import pandas as pd
from json import loads
from get_network import *

In [2]:
## for each page, extract followers/ following
def Follow(response):
    soup = BeautifulSoup(response, "html.parser")
    follow = soup.find_all("td", "info fifty screenname")
    cursor = soup.find_all("div", "w-button-more")
    try: cursor = findall(r'cursor=(.*?)">', str(cursor))[0]
    except IndexError: pass  
    follow = [i.find('a')['name'] for i in follow]
    return follow, cursor

In [3]:
## given a user, loop through its followers/ following pages using cursors
def scrape_username(username, fol = 'followers'):
    mobile = "https://mobile.twitter.com"
    url = f"{mobile}/{username}/{fol}?lang=en"
    init = '-1'
    follower_lst = []
    count = 0
    
    while(len(init)!=0):
        if init == '-1': tar_url = url
        else: tar_url = url + f"&cursor={init}"
        print(tar_url)
        r = requests.get(tar_url)
        if not r: break
        followers, init = Follow(r.text)
        follower_lst.extend(followers)
    return {username: follower_lst}

In [4]:
## deleted paul ryan and many other user who have too much followers
## used "check_follower" because I need to rescrape some users
data = pd.read_csv('twitter_accounts.csv')
sub_data = data[(data['check_follower']==1) & 
                (data['true_followers'] < 100000)]
accounts = list(sub_data['twitter'])
print(len(accounts))

37


In [6]:
#datasets = json.load(open('account_type.json'))
#expert_twitters = list(set(data['twitter']))
#tar_accounts = set([j for k in datasets for j in datasets[k]] + expert_twitters)
pwex = pywren.default_executor(job_max_runtime = 500)
#scrape_helper = lambda username: scrape_network(username, tar_accounts)
#futures = pwex.map(scrape_helper, accounts)
futures = pwex.map(scrape_username, accounts)
print(len(futures))

37


In [20]:
result = pywren.get_all_results(futures)
data = dict()
for num, item in enumerate(result):
    if num == 0: continue
    print(num, end = '\r')
    data.update(item.result())

In [21]:
## delete the data in Boto3
import boto3
def cleanup(bucket_name):
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(bucket_name)
    for item in bucket.objects.all(): item.delete()      
cleanup('chen.liang-pywren-478')

In [16]:
total_data = json.load(open('followers_parallel.json'))
for user in data: total_data.update({user: data[user]})
json.dump(total_data, open('followers_parallel.json', 'w'))

In [27]:
#total_data = json.load(open('following_parallel.json'))
#for user in data: total_data.update({user: data[user]})
#json.dump(data, open('following_parallel.json', 'w'))